In [1]:
# Chapter23
# 23.1 Exampleクラス
# 22.4 minkowski distance
def minkowskiDist(v1, v2, p):
    """ v1とv2は長さの等しい数値配列であるとする
        v1とv2の、p次のミンコウスキ距離を返す。
        p = 1: マンハッタン距離
        p = 2: ユークリッド距離"""
    dist = 0.0
    for i in range(len(v1)):
        dist += abs(v1[i]-v2[i])**p
    return dist**(1/p)

In [2]:
class Example(object):

    def __init__(self, name, features, label = None):
        """ name: string
            features: floatのリスト"""
        self.name = name
        self.features = features
        self.label = label
    
    # 次元数
    def dimensionality(self):
        return len(self.features)

    def getFeatures(self):
        return self.features[:] # コピーを返却
    
    def getLabel(self):
        return self.label
    
    def getName(self):
        return self.name
    
    # ユークリッド距離を返す
    def distance(sefl, other):
        return minkowskiDist(self.features, other.getFeatures(),2)
    
    def __str__(self):
        return self.name + ':' + str(self.features) + ':' + str(self.label)

In [4]:
# 23.2 Clusterクラス
class Cluster(object):

    def __init__(self, examples):
        """examples: Exampleの空でないリスト"""
        self.examples = examples
        self.centroid = self.computeCentroid()
    
    def update(self, examples):
        """examples: Exampleの空でないリスト
           examplesを更新し、重心の変化の総量を返す"""
        oldCentroid = self.centroid
        self.examples = examples
        self.centroid = self.computeCentroid()
        return oldCentroid.distance(self.centroid)
    
    def computeCentroid(self):
        vals = pylab.array([0.0]*self.examples[0].dimensionality())
        for e in self.examples: # 平均を計算
            vals += e.getFeature()
        centroid = Example('centroid', vals/len(self,examples))
        return centroid
    
    def getCentroid(self):
        return self.centroid
    
    def variability(self):
        totDist = 0.0
        for e in self.examples:
            totDist += (e.distance(self.centroid))**2 # クラスタ中心との距離を2乗
        return totDist
    
    def members(self):
        for e in self.examples:
            yield e
    
    def __str__(self):
        names = []
        for e in self.examples:
            names.append(e.getName())
        names.sort()
        result = 'Cluster with centroid' + str(self.centroid.getFeatures()) + 'contains:\n'
        for e in names:
            result = result + e + ', '
            return result[:-2] # 最後にあるコンマ・スペースを除去

In [5]:
# 23.4
import random
import pylab

def kmeans(examples, k, verbose = False):
    # 初期のk個のランダムな重心を選択
    # それぞれの重心に対してクラスタを形成
    initialCentroids = random.sample(expamples, k) # examplesからk個取得
    clusters = []
    for e in initialCentroids:
        clusters.append(Cluster([e]))
    
    # 重心が変化しなくなるまで繰り返す
    converged = False # 収束判定
    numIterations = 0
    while not converged:
        numIterations += 1
        # k個の空のリストからなるリストを作成する
        newClusters = []
        for i in range(k):
            newClusters.append([])
        
        # 標本それぞれeについて
        for e in examples:
            # eに最も近い重心を見つける
            smallestDistances = e.distance(clusters[0].getCentroid())
            index = 0
            for i in range(1,k):
                distance = e.distance(clusters[i].getCentroid())
                if distance < smallestDistances:
                    smallestDistances = distance
                    index = i
            # 適切なクラスタの標本リストにeを加える
            newClusters[index].append(e)
        
        for c in newClusters: # 空のクラスタを避ける
            if len(c) == 0:
                raise ValueError('Empty Cluster')
        
        # クラスタを更新する: 重心が変化しているかチェック
        converged = True
        for i in range(k):
            if clusters[i].update(newClusters[i]) > 0.0:
                convarged = False
        if verbose:
            print('Iteration #' + str(numIterations))
            for c in clusters:
                print(c)
            print('') # 空行を追加
    return clusters